<a href="https://colab.research.google.com/github/Akkuun/M1_IMAGINE_ML_Project/blob/main/ML_Project_Group_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<H1>Projet Machine Learning</H1>
<H2>HAI817I - 2024/2025</H2>
<H3>Classification d'assertions venant d‚ÄôX (Twitter) selon leur rapport √† la science </H3>



<H1>Groupe 8 </H1>
<br>

Duban Mathis: 22102226

Gonzalez Oropeza Gilles : 21602817

Bernardon Vincent : 22009116

LONGLADE Micka√´l :22105047




<H2> Sujet </H2>

Ce projet s‚Äôinscrit dans le contexte de l‚Äôapprentissage supervis√©, i.e. les donn√©es poss√®dent des
labels. Il vise √† trouver les mod√®les les plus performants pour pr√©dire si des assertions (une
assertion est une proposition que l'on avance et que l'on soutient comme vraie) faites par des
hommes politiques (par exemple) sont vraies ou fausses.




---



<H1>Tables des mati√®res </H1>



*   Installation





## **Installation**


**bold text**# Dans cette partie, nous allons installer toutes les librairies que nous allons avoir besoin pour notre projet.




In [2]:
# Installation des librairies pour le projet
!pip install pandas numpy emoji nltk matplotlib seaborn scikit-learn inflect googletrans==4.0.0-rc1 contractions

# Importation des diff√©rentes librairies utiles pour le notebook

import warnings # Supprime les warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Librairies de manipulation de donn√©es (graphique, lecture de donn√©es,ect...)
import pandas as pd # Lecture de donn√©es
import numpy as np # Array
import seaborn as sns #
import matplotlib.pyplot as plt # Graphique
import sys

# Librairies pour la fonction prepareText
import re # Regular expression
import nltk
from nltk.corpus import stopwords #English stopwords
nltk.download('stopwords') # T√©l√©chargement des stopwords (une seule fois)
from nltk.corpus import wordnet #Mots pour v√©rifier les suppressions de lettres r√©p√©t√©es
nltk.download('wordnet') # T√©l√©chargement de mots existants
import emoji
import inflect # Transformation des chiffres en mots
import re
from googletrans import Translator # Traduction de langues
from nltk.stem import WordNetLemmatizer # Lemmatisation des mots
from nltk.stem import PorterStemmer # Racinanisation des mots
nltk.download('punkt') #Tagetisation
nltk.download('averaged_perceptron_tagger')
from nltk import pos_tag # Tagination des mots
from nltk.tokenize import word_tokenize
from nltk import pos_tag
import unicodedata # Suppresion d'accent
import contractions # Transformation des contractions
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package ave

True

Importation du r√©pertoire de travail sur Google Drive


In [3]:
from google.colab import drive
drive.mount('/content/gdrive/')
path='/content/gdrive/My Drive/M1_IMAGINE_ML/ML_Project/'
sys.path.append(path)
%cd $path
%pwd

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
/content/gdrive/My Drive/M1_IMAGINE_ML/ML_Project


'/content/gdrive/My Drive/M1_IMAGINE_ML/ML_Project'

R√©cup√©ration des donn√©es du dataSet pr√©sent sur le r√©pertoire Google Drive en ligne


In [4]:
# Importation des donn√©es
df=pd.read_csv('dataSet/data.csv', sep='\t')
# Lecture des 5 premi√®res lignes pour confirmer la bonne r√©cup√©ration des donn√©es
display (df.head())

Unnamed: 0            tweet_id  \
0           0  316669998137483264   
1           1  319090866545385472   
2           2  322030931022065664   
3           3  322694830620807168   
4           4  328524426658328576   

                                                text  science_related  \
0  Knees are a bit sore. i guess that's a sign th...                0   
1          McDonald's breakfast stop then the gym üèÄüí™                0   
2  Can any Gynecologist with Cancer Experience ex...                1   
3  Couch-lock highs lead to sleeping in the couch...                1   
4  Does daily routine help prevent problems with ...                1   

   scientific_claim  scientific_reference  scientific_context  
0               0.0                   0.0                 0.0  
1               0.0                   0.0                 0.0  
2               1.0                   0.0                 0.0  
3               1.0                   0.0                 0.0  
4               1.0                   0.0                 0.0

R√©cup√©ration de nos dictionnaires :

In [5]:
# R√©cup√©ration des monaies
currency_file=pd.read_csv('dataSet/currency_translation.csv')
display (currency_file.head())
# Cr√©ation d'un objet dictionnaire pour r√©portorier les objets
currency_dict = dict(zip(currency_file['Currency Symbol'], currency_file['Currency Name']))
# R√©cup√©ration des abbr√©viations (Slang)
slang_file=pd.read_csv('dataSet/slang_translation.csv')
display (slang_file.head())
# Cr√©ation d'un objet dictionnaire pour r√©portorier les objets
slang_dict = dict(zip(slang_file['Abbreviation'], slang_file['Meaning']))

Currency Symbol Currency Name
0               $        dollar
1               ‚Ç¨          euro
2               ¬£         pound
3               ¬•           yen
4               ‚Çπ  indian rupee

Abbreviation                    Meaning
0          4ao            for adults only
1          a.m              before midday
2           a3  anytime anywhere anyplace
3        aamof        as a matter of fact
4         acct                    account

## **Ing√©nierie des donn√©es**

Dans cette partie, on s'inter√©ss√®e au pr√©-traitement des donn√©es. Afin que chaque √©l√©ments de notre base soit utilisable et pertinent on va nettoyer, normaliser et transformer nos donn√©es afin qu'elle soit pr√©par√©es et optimale pour nos analyses.


<br>

On va ici r√©pertorier tous les √©l√©ments qui doivent √™tre trait√© :

<br>


| √âl√©ment            | Exemple                 | Traitement √† effectuer | Exemple apr√®s traitement |
|--------------------|------------------------|-------------------------|--------------------------|
| Emojis            | üèÄ                     | Suppression des √©mojis ou remplacement par leur signification |  basketball |
| Mention Twitter   | @username               | Remplacement par un Token |  @MENTION |
| Hashtag           | #example                |  Remplacement par un Token | @HASHTAG |
| URL               | http://t.co/XGUfUDoLJB  | Suppression de l'URL | "" |
| Chiffre           | 13                      | Transformation en String | thirdteen |
| Majuscules        | Hello                    | Suppression de la majuscule | hello |
| Ponctuation       | !                        | Suppression de la ponctuation | "" |
| Mots r√©p√©t√©s      | cool cool cool         | Normalisation en une seule occurrence | cool |
| Lettres r√©p√©t√©es  | that's greeeeeeeat!        | R√©duction des r√©p√©titions excessives | that's great! |
| Abr√©viations      | ngl, fr | Remplacement par la version compl√®te | not gonna lie, for real |
| Stopwords (d√©terminants)         | the, and, a  | Suppression si non pertinent | "" |
| Slang (Argot)     | gonna, dunno, wanna           | Remplacement par des mots standards | going to, do not know, want to |
| Langue √©trang√®re  | bonjour, gracias          | D√©tection et traduction √©ventuelle | hello, thank you |
| Caract√®res sp√©ciaux | ¬ß, $, ^                | Suppression des charact√®res | "" |
| Expressions courantes | btw, lol    | Remplacement par la version compl√®te | by the way, laughing out loud|
| N√©gation mal format√©e | ain't, dunno          | Correction grammaticale | am not, do not know |
| √âmojis en Unicode | `\U0001F60D` (üòç)      | Conversion en texte lisible | smiling_face_with_hearteyes |
| Symboles de devises |    10\$, 10‚Ç¨               | Normalisation (ex: "10 euros") | 10 dollars, 10 euros |
| Accents |    clich√©              | Normalisation | cliche |






On cr√©er donc notre fonction permettant de preparer nos donn√©es brutes afin de les reformater correctement :

In [6]:
stop_words_set = set(stopwords.words('english'))
translator = Translator() # initialisation du traducteur
lemmatizer = WordNetLemmatizer() # initialisation du lematiseur
stemmer = PorterStemmer() # initialisation du taggeisteur


# Fonction permettant de pr√©parer la cha√Æne de charact√®res pass√© en param√®tre
def prepareText(text):
    #Majuscule, suppression
    data = str(text).lower()  # Convertir en minuscules

    #Suppresion d'accent
    data = remove_accents(data)

    #Contraction, on corrige
    data = contractions.fix(data)

    #Emoji, transformation en String
    data = emoji.demojize(data)

    #Mention Twitter, transformation en Token
    data = re.sub(r'@\w+', '@MENTION', data)

    #Hashtag, transformation en Token
    data = re.sub(r'#\w+', '@HASHTAG', data)

    #URL, on supprime
    data = re.sub(r'https?://\S+|www\.\S+', '', data)


    #Devise, remplacement par sa cha√Æne de charactere
    for symbol, name in currency_dict.items():
        data = re.sub(rf'(\d+){re.escape(symbol)}', r'\1 ' + name, data)

    #Chiffre, transformation en String
    words = data.split()
    data = ' '.join([inflect.engine().number_to_words(word) if word.isdigit() else word for word in words])

    #Ponctuation & charact√®res sp√©ciaux, suppression
    data = re.sub(r'[^\w\s]', '', data)

    #Stopwords, suppression
    data = ' '.join([word for word in data.split() if word not in stop_words_set])


    #Abr√©viation (Slang)
    data = ' '.join([slang_dict.get(word, word) for word in data.split()])

    #Mots r√©p√©t√©s
    data = re.sub(r'\b(\w+)(\s+\1\b)+', r'\1', data)

    #Lettres r√©p√©t√©s
    data = ' '.join([fixRepeat(word) for word in data.split()])

    #Traduction du tweet
    try:
        data = translator.translate(data, dest='en').text
    except:
        pass

    return data

#Fonction permet de supprmimer les r√©p√©titions succ√©sive de lettres (cas particulier rencontr√©)
def fixRepeat(word):
    # Reduce excessive repetition to exactly 2 occurrences
    repeat_regexp = re.compile(r'(\w*)(\w)\2{2,}(\w*)')
    repl = r'\1\2\2\3'

    if wordnet.synsets(word):
        return word

    repl_word = repeat_regexp.sub(repl, word)
    if repl_word != word:
        return fixRepeat(repl_word)

    # Try all combinations of removing one duplicate letter at a time
    candidates = set()
    for i in range(len(repl_word) - 1):
        if repl_word[i] == repl_word[i + 1]:
            candidates.add(repl_word[:i] + repl_word[i+1:])

    # Check if any of the candidates is a valid word
    for candidate in candidates:
        if wordnet.synsets(candidate):
            return candidate

    # No valid word is found, return the single-letter version
    single_letter_version = re.sub(r'(.)\1', r'\1', repl_word)
    return single_letter_version

# Fonction permettant de supprimer les accents
def remove_accents(text):
    return ''.join(c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn')


Exemple du passage de notre fonction


In [7]:
#URL
display("http://t.co/XGUfUDoLJB")
display(prepareText("http://t.co/XGUfUDoLJB"))
print("\n")
#Chiffre
display("3")
display(prepareText("3"))
print("\n")
#Majuscule
display("Hello")
display(prepareText("Hello"))
print("\n")
#Ponctuation
display("Hello!")
display(prepareText("Hello!"))
print("\n")
#Abr√©viation
display("lol")
display(prepareText("lol"))
print("\n")
#StepWord
display("After planning the project, she carefully researched each step, ensuring the execution was smooth and timely")
display(prepareText("After planning the project, she carefully researched each step, ensuring the execution was smooth and timely"))
print("\n")
#Emojis
display("üòç")
display(prepareText("üòç"))
print("\n")
#Traductions (derni√®re √©tape)
display("Êàë‰ªäÂ§©ÂéªË∂ÖÂ∏Ç‰π∞‰∫Ü‰∏Ä‰∫õÊ∞¥Êûú")
display(prepareText("Êàë‰ªäÂ§©ÂéªË∂ÖÂ∏Ç‰π∞‰∫Ü‰∏Ä‰∫õÊ∞¥Êûú"))
print("\n")
#Mention Twitter
display("as @username said it's bad !")
display(prepareText("as @username said it's bad !"))
print("\n")
#Hashtag
display("I went to the theater to see Dune 2 #Dune")
display(prepareText("I went to the theater to see Dune 2 #Dune"))
print("\n")
#Charact√®res sp√©ciaux
display("¬ß$¬£")
display(prepareText("¬ß$¬£"))
print("\n")
#Devices
display("10$ 10¬£ 10‚Ç¨")
display(prepareText("10$ 10¬£ 10‚Ç¨"))
print("\n")
#Mot r√©p√©t√©s
display("Cool Cool Cool Cool Hot Hot Hot")
display(prepareText("Cool Cool Cool Cool Hot Hot Hot"))
print("\n")
#Lettres r√©p√©t√©es
display("Steaaaaaaaak tendeeeeers beeeeer goooooose   threeeeeeee woooooooooood aggggggggressive")
display(prepareText("Steaaaaaaaak tendeeeeers beeeeer goooooose   threeeeeeee woooooooooood aggggggggressive"))
print("\n")
#Accent
display("clich√©")
display(prepareText("clich√©"))

'http://t.co/XGUfUDoLJB'

''

'3'

'three'

'Hello'

...

'cool hot'

'Steaaaaaaaak tendeeeeers beeeeer goooooose   threeeeeeee woooooooooood aggggggggressive'

'steak tenders beer goose three wood aggressive'

'clich√©'

'cliche'

On cr√©√© une copie de notre set de donn√©es de base et on applique notre fonction sur tout nos tweets

In [8]:
# Cr√©ation copie donn√©es
dataPrepared =df.copy()
# Application de notre fonction de preparation sur dataPrepared
dataPrepared['text'] = dataPrepared['text'].apply(prepareText)
# T√©l√©chargement CSV
dataPrepared.to_csv('dataSet/dataPrepared.csv', index=False)


On supprime toutes les lignes contenant un tweet vide :

In [ ]:
# Suppression de toutes les lignes vides


Une fois notre premier traitement effectu√© on va effecter la derni√®re partie des traitements des donn√©es brutes la lemmatisation, racinisation et tagination. Ces √©tapes permettent d'affiner le texte pour que chaque mot soit r√©duit √† sa forme de base, ce qui est essentiel pour de nombreuses applications de traitement de texte, comme la recherche d'informations ou l'analyse de sentiments.

Voici les √©tapes que l'on va faire apr√®s le formatage :

Lemmatisation : Cette technique consiste √† r√©duire un mot √† sa forme canonique (ou lemm√©), c'est-√†-dire √† la forme sous laquelle il appara√Æt dans le dictionnaire. Exemple better deviendra good.

Racinisation : Cette m√©thode consiste √† r√©duire un mot √† sa racine, c'est-√†-dire √† enlever les suffixes (ou pr√©fixes) pour obtenir une forme simplifi√©e du mot. Cela permet de mieux traiter les variations de mot comme runner qui devriendra run.

Tagination (ou √©tiquetage de parties du discours) : Cette technique consiste √† identifier et √† √©tiqueter chaque mot d'un texte en fonction de sa cat√©gorie grammaticale (nom, verbe, adjectif, etc.)

In [ ]:
#application sur  dataPrepared